In [1]:
import syft as sy
from syft import ActionObject
from syft import syft_function, syft_function_single_use
from time import sleep
import os
import psutil
import inspect

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft/notebooks


with server

In [2]:
node = sy.orchestra.launch(name="test-domain-helm2", dev_mode=True,
                           reset=True, 
                           n_consumers=3,
                           create_producer=True
                          )
                           
client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/448cb501cca248c7b9845081ed40744c.sqlite

Consumer service Name:  None
Creating Default Worker Image
Building Default Worker Image
Creating default Worker Pool
Created default worker pool.
Data Migrated to latest version !!!
Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [3]:
# res = client.register(name="a", email="b@b.org", password="c", password_verify="c")

# res = client.register(name="a", email="aa@b.org", password="c", password_verify="c")

# ds_client = node.login(email="aa@b.org", password="c")

# ds_client2 = node.login(email="b@b.org", password="c")

# client.users

setup: compute train-test overlap between a very large train set and a smaller test set. Small test is still to big for memory, so we split it into 54 parts. We keep 1 of those parts in memory. We dont keep the train set in memory, but read and compare with 1/54 parts line by line. Each part takes ~30 hours, but we can run 54 processes in parallel.

# Setup syft functions

## Dataset

In [4]:
x = ActionObject.from_obj([1, 2])
x_ptr = x.send(client)

## Batch function

In [5]:
@syft_function()
def process_batch(batch):
    # takes 30 hours normally
    print(f"starting batch {batch}")
    from time import sleep
    sleep(1)
    print("done")
    return batch+1

SyftSuccess: Syft function 'process_batch' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [6]:
client.code.submit(process_batch)

SyftSuccess: User Code Submitted

## Main function

In [7]:
@syft_function_single_use(x=x_ptr)
def process_all(domain, x):
    jobs = []
    print("Launching jobs")
    for elem in x:
        # We inject a domain object in the scope
        batch_job = domain.launch_job(process_batch, batch=elem)
        jobs += [batch_job]
    print("starting aggregation")
    print("Done")
    return 3

SyftSuccess: Syft function 'process_all' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

# Approve & run

In [8]:
# client.code.submit(process_all)

In [9]:
req = client.code.request_code_execution(process_all)


In [10]:
req

syft.service.request.request.Request

In [12]:
req.approve(approve_nested=True)

Approving request for domain test-domain-helm2


SyftSuccess: Request 776e30f213c6451390f4a0a2aa9dc55e changes applied

In [13]:
# client._fetch_api(client.credentials)

In [22]:
job = client.code.process_all(x=x_ptr, blocking=False)

29/01/24 18:08:07 FUNCTION LOG (e3fbdc575f4d4426b9a86142075766ff): Launching jobs
29/01/24 18:08:07 EXCEPTION LOG (e3fbdc575f4d4426b9a86142075766ff):

Encountered while executing process_all:
Traceback (most recent call last):
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1347, in execute_byte_code
    result = eval(evil_string, _globals, _locals)  # nosec
  File "<string>", line 1, in <module>
  File "<string>", line 12, in user_func_process_all_47f8c8f3db3a30695a28e4a51e44916669ac3d111924cb614181c64b2c3b8323_e18f2bd3f95b6f8df70995b4138a72d51e65846799577546427ef18b8cf22e61
  File "<string>", line 7, in process_all
NameError: name 'process_batch' is not defined

    4      print('Launching jobs')
    5      for elem in x:
--> 6          batch_job = domain.launch_job(process_batch, batch=elem)
    7          jobs += [batch_job]


In [23]:
job

```python
class Job:
    id: UID = e3fbdc575f4d4426b9a86142075766ff
    status: errored
    has_parent: False
    result: Err('Exception encountered while running process_all, please contact the Node Admin for more info.')
    logs:

0 Launching jobs
1 
2 
3 Encountered while executing process_all:
4 Traceback (most recent call last):
5   File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1347, in execute_byte_code
6     result = eval(evil_string, _globals, _locals)  # nosec
7   File "<string>", line 1, in <module>
8   File "<string>", line 12, in user_func_process_all_47f8c8f3db3a30695a28e4a51e44916669ac3d111924cb614181c64b2c3b8323_e18f2bd3f95b6f8df70995b4138a72d51e65846799577546427ef18b8cf22e61
9   File "<string>", line 7, in process_all
10 NameError: name 'process_batch' is not defined
11 
12     4      print('Launching jobs')
13     5      for elem in x:
14 --> 6          batch_job = domain.launch_job(process_batch, batch=elem)
15     7          jobs += [batch_job]
    
```